In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))

In [3]:

%run ../load_magic/storage.py
from cypher_utils import CypherUtilities
from datetime import datetime
from ha_utils import HeaderAnalysis
from scrape_utils import WebScrapingUtilities
from tqdm.notebook import tqdm
import humanize
import time
import winsound

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)
duration = 1000  # milliseconds
freq = 880  # Hz
print(f'Last run on {datetime.now()}')

Last run on 2023-01-16 08:07:35.806914


In [8]:

t0 = time.time()
verbose = False
cypher_files_list = list(set(cu.get_files_list(verbose=verbose)))
files_list = sorted([fn for fn in os.listdir(cu.SAVES_HTML_FOLDER) if fn.endswith('.html') and (fn not in cypher_files_list)])
pbar = tqdm(files_list, total=len(files_list))
strings_added = 0
for file_name in files_list:
    pbar.set_description('{:<50}'.format(file_name.split('.')[0].replace('_Indeed_com', '')[-50:]))
    cu.ensure_filename(file_name, verbose=verbose)
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    if not row_div_list:
        row_div_list = page_soup.find_all(name='body')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=verbose)
        strings_added += len(child_strs_list)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
wsu.beep(freq, duration)
print(f'{strings_added} child string tag sequences populated in {duration_str}')

0it [00:00, ?it/s]

0 child string tag sequences populated in 0 seconds
